In [1]:
from mindspore import Tensor, dtype as mstype, Parameter, save_checkpoint
from timm.models import create_model
checkpoint_path = './L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_384.npz'

In [ ]:
prefix = "model.encoder."
model = create_model("vit_large_patch16_384", img_size=768, pretrained=True, checkpoint_path=checkpoint_path)

In [ ]:
state_dict = model.state_dict()

weights = []
prefix = "model.encoder."
for key in state_dict:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        name2weight["data"] = Parameter(Tensor(state_dict[key].numpy(), mstype.float32),requires_grad=True)
        weights.append(name2weight)
    else:
        dd = mstype.float32
        weight = Parameter(Tensor(state_dict[key].numpy(), dd),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})
    print(weights[-1]["name"], weights[-1]["data"].shape)

In [ ]:
save_checkpoint(weights, "vit_large_patch16_384.ckpt")